In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from skimage.transform import resize
from tqdm import tqdm
from keras import backend as K
from torchvision import models
import torch
from PIL import Image

from visualize import *
import cv2

In [ ]:
def generate_masks(N, s, p1):
    cell_size = np.ceil(np.array(model.input_size) / s)
    up_size = (s + 1) * cell_size

    grid = np.random.rand(N, s, s) < p1
    grid = grid.astype('float32')

    masks = np.empty((N, *model.input_size))

    for i in tqdm(range(N), desc='Generating masks'):
        # Random shifts
        x = np.random.randint(0, cell_size[0])
        y = np.random.randint(0, cell_size[1])
        # Linear upsampling and cropping
        masks[i, :, :] = resize(grid[i], up_size, order=1, mode='reflect',
                                anti_aliasing=False)[x:x + model.input_size[0], y:y + model.input_size[1]]
    masks = masks.reshape(-1, *model.input_size, 1)
    return masks

batch_size = 100

def explain(model, inp, masks):
    preds = []
    # Make sure multiplication is being done for correct axes
    masked = inp * masks
    for i in tqdm(range(0, N, batch_size), desc='Explaining'):
        preds.append(model.run_on_batch(masked[i:min(i+batch_size, N)]))
    preds = np.concatenate(preds)
    sal = preds.T.dot(masks.reshape(N, -1)).reshape(-1, *model.input_size)
    sal = sal / N / p1
    return sal

In [ ]:
# from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
# from keras import backend as K
from keras.preprocessing import image

class Model():
    def __init__(self):
      K.set_learning_phase(0)
      self.model = models.vgg16(pretrained=True)
      # self.model = VGG16(weights='imagenet')
      self.input_size = (224, 224)

    def run_on_batch(self, x):
      print('run on batch')
      # x = torch.Tensor(x)      
      # return self.model(x)
      return self.model.predict(x)

def load_img(path):
  # img = image.load_img(path)
  img = cv2.imread(path)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  return img, x

In [ ]:
imagePath = []
targets = [58,222,332,417,429,429,525,549,683,889,990]
features_ls = [30,28,26,24,21,19,17,14,12,10,7,5,2,0]

trained = Model()
untrained = models.vgg16(pretrained=False)

for i, path in enumerate(imagePath):
  N = 2000
  s = 8
  p1 = 0.5
  masks = generate_masks(N, s, p1)

  img, x = (path)
  
  for layer in features_ls:
    if layer == 28:
      trained.classifier = untrained.classifier
    if layer < 30:
      trained.features[layer] = untrained.features[layer]

    sal = explain(trained, x, masks)

    save_path = ''
    save_result(sal, save_path)

compute SSIM score

In [ ]:
base_path = ''

ssim_ls = compute_ssim(args.image_path, base_path) 

In [ ]:
plot_ssim(ssim_ls, 'RISE', '')